In [1]:
import scanpy as sc
import pandas as pd
# from scipy.io import mmread
import numpy as np
import scipy.sparse as sp
import os
import anndata as ad

# from sklearn.metrics import adjusted_rand_score
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import scanpy.external as sce  # Harmony 在 sc.external
# from sklearn.decomposition import PCA
import harmonypy as hm
import warnings
warnings.simplefilter("ignore", category=pd.errors.PerformanceWarning)

In [3]:
adata = sc.read_h5ad("./adata_05a_with_embeddings.h5ad")
adata

AnnData object with n_obs × n_vars = 49641 × 27187
    obs: 'prject', 'nUMI', 'nGene', 'cellID', 'Doublet_Score', 'Doublet_Class', 'log10GenesPerUMI', 'mitoRatio', 'riboRatio1', 'riboRatio2', 'riboRatio3', 'riboRatio4', 'riboRatio', 'RNA_snn_res.0', 'RNA_snn_res.0.1', 'RNA_snn_res.0.2', 'RNA_snn_res.0.3', 'RNA_snn_res.0.4', 'RNA_snn_res.0.5', 'RNA_snn_res.0.6', 'RNA_snn_res.0.7', 'RNA_snn_res.0.8', 'RNA_snn_res.0.9', 'RNA_snn_res.1', 'seurat_clusters', 'class1', 'sample', 'source', 'S.Score', 'G2M.Score', 'Phase', 'RNA_snn_res.1.2', 'RNA_snn_res.1.4', 'RNA_snn_res.1.6', 'RNA_snn_res.1.8', 'RNA_snn_res.2', 'cellType', 'dcellType', 'cell', 'patient', 'timing', 'category', 'response', 'sub.response', 'date', 'start', 'daysSinceStart', 'blast', 'age', 'sex', 'FAB', 'cellCount', 'monthsSinceStart', 'relapse', 'daysStartRelapse', 'monthsStartRelapse', 'azimuthNames', 'mut', 'p53.mut', 'n_counts', 'venetoclax', 'treat'
    var: 'vst.mean', 'vst.variance', 'vst.variance.expected', 'vst.varianc

In [ ]:

adata_ctrl = adata[adata.obs["patient"].isin(["HCBM1","HCBM2","HCBM3"])].copy()
adata_aml = adata[~adata.obs["patient"].isin(["HCBM1","HCBM2","HCBM3"])].copy()

In [ ]:
hierarchical_markers = {
    "CD4": {
        # "general": ["CD4", "IL7R", "MAL", "LEF1", "LTB", "LDHB", "TPT1", "TRAC", "TCF7", "TMSB10", "LEPROTL1","CD3D", "CD3E","CD3G", "CD27"],   # broad CD4 T cell markers
         "general": ["CD4", "IL7R", "MAL", "LTB"],   # broad CD4 T cell markers
        "subtypes": {
            "CD4 Memory T cells": ["CXCR3", "IL7R", "LTB"],
            "CD4 Naive T cells": ["LEF1","TCF7","SELL","CD44"], #Naive "IL7R", "LTB"
            "CD4 CTL T cells": ["GZMB","PRF1","GNLY","NKG7"],
            "CD4 Exhausted T cells": ["CTLA4","LAG3","TIGIT","HAVCR2","PDCD1"],
            "CD4 Th1 T cells": ["TBX21","STAT4","IFNG","IL12A"],
            "CD4 Th2 T cells": ["STAT6","GATA6","IL4"],
            "CD4 Th17 T cells": ["IL7RA","STAT3","RORC"],
            "CD4 Tfh T cells": ["BCL6","CXCR5"],
            "Treg": ["TGFB1","FOXP3","IL2RA","IKZF2"]
        }
    },
    "CD8": {
        # "general": ["CD8A","CD8B","NELL2", "CD3D", "CD3E", "S100B", "GZMH", "CD3G", "TRGC2", "CCL5"],
        "general": ["CD8A","CD8B", "CD3D" ],
        "subtypes": {
            "CD8 Naive T cells": ["NELL2","CD8B","CCR7","LEF1","SELL","IL7R"],
            "CD8 Effector Memory T cells": ["PRF1","FGFBP2","FCGR3A","KLRD1","NKG7","KLRG1","CX3CR1"],
            "CD8 Exhausted T cells": ["CTLA4","LAG3","TIGIT","PDCD1","TOX"],
            "CD8 CTL T cells": ["GZMB","PRF1","GNLY","NKG7","GZMA"],
            "MAIT": ["KLRB1","IL7R","SLC4A10","RORC"]
        }
    },
    # "B": {
    #     "general": ["FCRL1", "FCRL2", "CD22", "ARHGAP24", #Azimuth
    #                                                            "BANK1", "MS4A1", "RALGPS2", "CD37",
    #                                                            "SWAP70", "CD79A",
    #                                                            "CD19", "B220"],#Abbas Lab
    #     "subtypes": {}
    # },
    "B": {
        "general": ["MS4A1", "CD79A", "CD79B", "BANK1", "RALGPS2"],
        "subtypes": {
            "B intermediate": ["MS4A1", "TNFRSF13B", "IGHM", "IGHD", "AIM2", "CD79A", "LINC01857", "RALGPS2", "BANK1", "CD79B"],
            "B memory": ["MS4A1", "COCH", "AIM2", "BANK1", "SSPN", "CD79A", "TEX9", "RALGPS2", "TNFRSF13C", "LINC01781"],
            "B naive": ["IGHM", "IGHD", "CD79A", "IL4R", "MS4A1", "CXCR4", "BTG1", "TCL1A", "CD79B", "YBX3"],
            "Plasmablast": ["IGHA2", "MZB1", "TNFRSF17", "DERL3", "TXNDC5", "TNFRSF13B", "POU2AF1", "CPNE5", "HRASLS2", "NT5DC2"]
        }
    },

    "pre B": {
        "general": ["NPY", "LCN6", "RAG2", "HMHB1",
                                                               "ARPP21", "AKAP12", "RAG1", "C10orf10",
                                                               "CYGB", "SLC8A1-AS1"],#Abbas Lab
        "subtypes": {}
    },

    # ---- Myeloid / Mono / Macrophage ----
    "Mono": {
        "general": ["LYPD2", "FOLR3", "CLEC4E", "LILRA1",
                    "CDA", "RBP7", "CD300LF", "FPR1", "CD93", "MTMR11"],
        "subtypes": {
            "CD14 Mono": ["FOLR3", "CLEC4E", "MCEMP1", "RBP7",
                          "CDA", "FPR1", "CD300E", "C5AR1",
                          "CD93", "APOBEC3A"],
            "CD16 Mono": ["LYPD2", "VMO1", "TPPP3", "C1QA",
                          "C5AR1", "CD300E", "GPBAR1", "LILRA1",
                          "HES4", "APOBEC3A"]
        }
    },
    # ===== NK lineage =====
    "NK": {
        "general": ["TRDC", "FCER1G", 
                    "KLRF1" 
                    ],
        # "general": ["NKG7", "KLRC1", "KLRD1", "KLRB1", 
        #                                                        "KIR2DL1", "KIR3DL1", "KIR2DL3", "KIR3DL2",
        #                                                        "KIR3DL3", "GNLY", "NCAM1", "FCGR3A"],
        "subtypes": {
            "NK CD56-dim": ["GNLY", "TYROBP", "NKG7", "GZMB",  "PRF1", "FGFBP2", "SPON2"],
            "NK Proliferating": ["MKI67" , "TYMS",  "TOP2A", "PCLAF", "CD247", "CLSPN", "ASPM"],
            "NK CD56-bright": ["XCL2",  "SPINK2", "KLRC1", "XCL1", "SPTSSB", "PPP1R9A", "NCAM1", "TNFRSF11A"]
        }
    },
    "EMPs": {
        "general": ["MYCT1", "CRHBP", "NPR3", "AVP", "HPGDS", "CRYGD","IGSF10", "PBX1", 
                    "CYTL1","GATA2"
                    ],
        
        "subtypes": {
            "Megakaryocyte": [
                "GFI1B", "SELP", "GP1BA", "CD9", "ITGA2B", "GATA2", "FLI1",
                "GP1BB", "VWF", "THPO", "ELF1", "THBS1", "MPIG6B", "GP9",
                "F2R", "FOG1", "NFE2", "SPI1", "PF4"
            ],
            "Erythroid progenitor": [
                "GATA1", "KLF1", "FCER1A", "ITAG2B", "EPOR", "HBD", "ZFPM1",
                "GATA2", "GYPA", "TFRC", "TFR2", "CSF2RB", "APOE", "APOC1",
                "CNRIP1", "FOXO3", "ETS1", "BRD1", "TAL1"
            ]
        }
    },

    # ---- DC ----
    "DC": {
        "general": ["CLEC4C", "PROC", "SCT", "SCN9A",
                    "SHD", "PPM1J", "ENHO", "CLEC10A",
                    "LILRA4", "DNASE1L3"],
        "subtypes": {
            'ASDC':['PPP1R14A', 'LILRA4', 'AXL', 'IL3RA', 'SCT', 'SCN9A', 'LGMN', 'DNASE1L3', 'CLEC4C', 'GAS6'],
            'cDC1':['CLEC9A', 'DNASE1L3', 'C1orf54', 'IDO1', 'CLNK', 'CADM1', 'FLT3', 'ENPP1', 'XCR1', 'NDRG2'],
            'cDC2':['FCER1A', 'HLA-DQA1', 'CLEC10A', 'CD1C', 'ENHO', 'PLD4', 'GSN', 'SLC38A1', 'NDRG2', 'AFF3'],
            'pDC':['ITM2C', 'PLD4', 'SERPINF1', 'LILRA4', 'IL3RA', 'TPM2', 'MZB1', 'SPIB', 'IRF4', 'SMPD3']
        }
    },

    # ---- Progenitors / Stem cells ----
    "HSC": {
        "general": ["CD34", "AVP", "CRHBP"],
        "subtypes": {}
    },
    "CLP": {
        "general": ["ACY3", "PRSS2", "C1QTNF4", "SPINK2",
                    "SMIM24", "NREP", "CD34", "DNTT", "FLT3", "SPNS3"],
        "subtypes": {}
    },
    "GMP": {
        "general": ["SERPINB10", "RNASE3", "MS4A3", "PRTN3",
                    "ELANE", "AZU1", "CTSG", "RNASE2", "RETN", "NPW"],
        "subtypes": {}
    },

    # ---- Erythroid / Megakaryocyte ----
    # "Early Erythroid": {
    #     "general": [
    #         "CNRIP1",  "ITGA2B", "TFR2", "MAP7", "FSCN1", "APOC1" "GATA1", "KLF1", 
    #                 "CYTL1","GATA2",
    #                 ],
    #     "subtypes": {}
    # },
    # "Late Erythroid": {
    #     "general": ["CTSE", "TSPO2", "IFIT1B", "TMEM56",
    #                            "RHCE", "RHAG", "SPTA1", "ADD2",
    #                            "EPCAM", "HBG1"],
    #     "subtypes": {}
    # },
    "Macrophage": {
        "general": ["SPIC", "FABP3", "CD5L", "CCL18",
                    "C1QC", "C1QB", "FABP4", "C1QA",
                    "APOE", "SELENOP"],
        "subtypes": {}
    },
    "Erythroid": {
        "general": ['HBD', 'HBM', 'AHSP', 'ALAS2','CA1', 'SLC4A1', 'IFIT1B', 'TRIM58', 'SELENBP1', 'TMCC2'],
        "subtypes": {
            "Early Erythroid": ["CNRIP1", "GATA2", "ITGA2B", "TFR2",
                                "GATA1", "KLF1", "CYTL1", "MAP7",
                                "FSCN1", "APOC1"],
            "Late Erythroid": ["CTSE", "TSPO2", "IFIT1B", "TMEM56",
                               "RHCE", "RHAG", "SPTA1", "ADD2",
                               "EPCAM", "HBG1"]
        }
    },
    "Platelets": {
        "general": ["RGS18", "C2orf88", "TMEM40", "GP9",
                    "PF4", "PPBP", "DAB2", "SPARC",
                    "RUFY1", "F13A1"],
        "subtypes": {}
    }
}

In [ ]:
celltype_colors_dict = {
    # ---- CD4 T ----
    "CD4": "#8dd3c7",
    "CD4 Memory T cells": "#66c2a5",
    "CD4 Naive T cells": "#99d8c9",
    "CD4 CTL T cells": "#41ae76",
    "CD4 Exhausted T cells": "#238b45",
    "CD4 Th1 T cells": "#005824",
    "CD4 Th2 T cells": "#b2e2e2",
    "CD4 Th17 T cells": "#7bccc4",
    "CD4 Tfh T cells": "#2ca25f",
    "Treg": "#006d2c",

    # ---- CD8 T ----
    "CD8": "#fb8072",
    "CD8 Naive T cells": "#fdbb84",
    "CD8 Effector Memory T cells": "#e34a33",
    "CD8 Exhausted T cells": "#b30000",
    "CD8 CTL T cells": "#f16913",
    "MAIT": "#fb6a4a",

    # ---- B cells ----
    "B": "#80b1d3",
    "B intermediate": "#4eb3d3",
    "B memory": "#2b8cbe",
    "B naive": "#7fcdbb",
    "Plasmablast": "#1c9099",

    # ---- pre B ----
    "pre B": "#08306b",

    # ---- Monocytes ----
    "Mono": "#bebada",
    "CD14 Mono": "#9e9ac8",
    "CD16 Mono": "#756bb1",

    # ---- NK ----
    "NK": "#fccde5",
    "NK CD56-dim": "#fa9fb5",
    "NK Proliferating": "#dd3497",
    "NK CD56-bright": "#980043",

    # ---- EMPs ----
    "EMPs": "#fdb462", 
    "Megakaryocyte": "#f47c3c",
    "Erythroid progenitor": "#fed98e",

    # ---- DC ----
    "DC": "#b3de69",
    "ASDC": "#66c2a5",
    "cDC1": "#31a354",
    "cDC2": "#006d2c",
    "pDC": "#b2df8a",

    # ---- Progenitors / Stem cells ----
    "HSC": "#bc80bd",
    "CLP": "#8c6bb1",
    "GMP": "#88419d",

    # ---- Macrophage ----
    "Macrophage": "#ffed6f",

    # ---- Erythroid ----
    "Erythroid": "#d9d9d9",
    "Early Erythroid": "#bdbdbd",
    "Late Erythroid": "#969696",


    # ---- Erythroid ----
    # "Erythroid": "#0b0a0a",
    # "Early Erythroid": "#0b0606",
    # "Late Erythroid": "#110303",

    # ---- Platelets ----
    "Platelets": "#a6cee3",

    # ---- Default ----
    "Unknown": "#999999"
}

# plt.figure(figsize=(8, 12))
# plt.axis("off")
# patches = [mpatches.Patch(color=color, label=ctype) for ctype, color in celltype_colors_dict.items()]

# plt.legend(
#     handles=patches,
#     loc='center left',
#     bbox_to_anchor=(0, 0, 1, 1),  # 左对齐，居中
#     ncol=2,                       # 分两列显示
#     fontsize=9,
#     title="Cell Type Color Legend",
#     title_fontsize=11,
#     frameon=False
# )

# plt.tight_layout()
# plt.show()
# plt.savefig("celltype_color_legend.png", dpi=300, bbox_inches="tight")


In [ ]:
def main_cluster(adata_filter, n_neighbors, embs,resolution,threshold_main,threshold_sub,celltype_colors_dict):
    # for key in ['X_harmony', 'X_pca', 'X_tsne', 'X_pca_harmony']:
    #     adata_filter.obsm.pop(key, None)

    # Z = adata_filter.obsm[embs]
    # Z = Z.toarray() if hasattr(Z, 'toarray') else Z  # 转为 dense

    # ho = hm.run_harmony(Z, adata_filter.obs, vars_use=['patient'])
    # # harmonypy 返回是 (dims x cells)，所以转置
    # adata_filter.obsm[f'{embs}_harmony'] = ho.Z_corr.T

    sce.pp.harmony_integrate(adata_filter, key="patient", basis=embs)
    sc.pp.neighbors(adata_filter, use_rep=f'X_harmony', n_neighbors=n_neighbors)
    sc.tl.leiden(adata_filter, resolution=resolution)
    # ari3 = adjusted_rand_score(adata_filter.obs["dcellType"], adata_pre.obs["leiden"])
    # print("Harmony on embedding ARI:", ari3)
    cluster_annotations = {}
    cluster_topN = {}

    sc.tl.rank_genes_groups(adata_filter, 'leiden', method='wilcoxon')

    cluster_means = adata_filter.to_df().groupby(adata_filter.obs['leiden']).mean()
    cluster_means = (cluster_means - cluster_means.mean()) / cluster_means.std()
    for cluster in cluster_means.index:
        lineage_scores = {}
        
        # ---- broad lineage ----
        for lineage, info in hierarchical_markers.items():
            general_markers = [g for g in info["general"] if g in cluster_means.columns]
            if not general_markers:
                continue
            lineage_scores[lineage] = cluster_means.loc[cluster, general_markers].mean()
        
        if not lineage_scores:
            cluster_annotations[cluster] = "Unknown"
            continue
        best_lineage, best_lineage_score = max(lineage_scores.items(), key=lambda x: x[1])
        
        if best_lineage_score < threshold_main:
            cluster_annotations[cluster] = "Unknown"
            continue
        
        # ---- sub ----
        subtype_scores = {}
        subtypes = hierarchical_markers[best_lineage]["subtypes"]
        
        if subtypes:
            for subtype, markers in subtypes.items():
                markers_in_data = [g for g in markers if g in cluster_means.columns]
                if not markers_in_data:
                    continue
                subtype_scores[subtype] = cluster_means.loc[cluster, markers_in_data].mean()
        
        if not subtype_scores:
            cluster_annotations[cluster] = best_lineage
        else:
            best_subtype, best_sub_score = max(subtype_scores.items(), key=lambda x: x[1])
            if best_sub_score >= threshold_sub:
                cluster_annotations[cluster] = best_subtype
            else:
                cluster_annotations[cluster] = best_lineage
        
        cluster_topN[cluster] = {
            "lineage": (best_lineage, best_lineage_score),
            "lineage_scores": lineage_scores,  
            "top_subtypes": sorted(subtype_scores.items(), key=lambda x: x[1], reverse=True)[:3]
    }

    adata_filter.obs['celltype'] = adata_filter.obs['leiden'].map(cluster_annotations)
    adata_filter.obs['celltype'] = adata_filter.obs['celltype'].astype('category')
    # adata_filter.uns['celltype_colors'] = [
    #     celltype_colors_dict.get(c, "#CCCCCC")  
    #     for c in adata_filter.obs['celltype'].cat.categories
    #     ]
    topN_summary = pd.DataFrame({
        k: {
            **{
                f"main_type_{i+1}": sorted(v["lineage_scores"].items(), key=lambda x: x[1], reverse=True)[i][0]
                if len(v["lineage_scores"]) > i else None
                for i in range(3)
            },
            **{
                f"main_score_{i+1}": sorted(v["lineage_scores"].items(), key=lambda x: x[1], reverse=True)[i][1]
                if len(v["lineage_scores"]) > i else None
                for i in range(3)
            }
        }
        for k, v in cluster_topN.items()
    }).T

    return adata_filter, cluster_topN, topN_summary 

In [2]:
import scanpy as sc
import pandas as pd
from itertools import combinations

group_files = {
    # ===== HC =====
    # "HC_EMP":            "adata_HC_EMP.h5ad",
    "HC_HSC":            "adata_HC_HSC.h5ad",
    # "HC_NK":             "adata_HC_NK.h5ad",
    # "HC_GMP1":            "adata_HC_GMP1.h5ad",
    # "HC_cDC1":           "adata_HC_cDC1.h5ad",
    # "HC_macrophage3":    "adata_HC_macrophage3.h5ad",
    # "HC_Erythroid(right)":"adata_HC_Erythroid.h5ad",

    # ===== pre_res =====
    "pre_res_MPPCLP1":   "adata_pre_res_MPPCLP1.h5ad",
    "pre_res_HSCLSC":    "adata_pre_res_HSCLSC.h5ad",
    # "pre_res_EMP":       "adata_pre_res_EMP.h5ad",
    # "pre_res_MPPCLP2":   "adata_pre_res_MPPCLP2.h5ad",
    # "pre_res_LE":        "adata_pre_res_LE.h5ad",
    # "pre_res_GMP1":      "adata_pre_res_GMP1.h5ad",
    # "pre_res_GMP2":      "adata_pre_res_GMP2.h5ad",
    # "pre_res_macrophage2":"adata_pre_res_macrophage2.h5ad",
    # "pre_res_Erythroid(right)":"adata_pre_res_Erythroid.h5ad",

    # ===== pre_nres =====
    "pre_nres_HSCLSClow":   "adata_pre_nres_HSCLSClow.h5ad",
    "pre_nres_HSCLSChigh":   "adata_pre_nres_HSCLSChigh.h5ad",
    "pre_nres_MPPCLP1":  "adata_pre_nres_MPPCLP1.h5ad",
    # "pre_nres_EMP":      "adata_pre_nres_EMP.h5ad",
    # "pre_nres_LE":       "adata_pre_nres_LE.h5ad",
    # "pre_nres_Erythroid(right)":"adata_pre_nres_Erythroid.h5ad",
    
    # ===== post_res =====
    # "post_res_EMP":      "adata_post_res_EMP.h5ad",
    # "post_res_LE":       "adata_post_res_LE.h5ad",
    # "post_res_GMP1":     "adata_post_res_GMP1.h5ad",
    # "post_res_Erythroid(right)":"adata_post_res_Erythroid.h5ad",

    # ===== post_nres =====
    "post_nres_ASDC":        "adata_post_nres_ASDC.h5ad",
    "post_nres_MPPpreB":     "adata_post_nres_MPPpreB.h5ad",
    "post_nres_HSCLSC":      "adata_post_nres_HSCLSC.h5ad",
    "post_nres_MPPCLP1":      "adata_post_nres_MPPCLP1.h5ad",
    # "post_nres_EMP":         "adata_post_nres_EMP.h5ad",
    # "post_nres_Macrophage1": "adata_post_nres_Macrophage1.h5ad",
    # "post_nres_LE":          "adata_post_nres_LE.h5ad",
    # "post_nres_GMP1":        "adata_post_nres_GMP1.h5ad",
    # "post_nres_Erythroid(right)":"adata_post_nres_Erythroid.h5ad",
}

# 读入
adatas = {g: sc.read_h5ad(f) for g, f in group_files.items()}


In [9]:

HSC_files = {
    "HC_HSC": "adata_HC_HSC.h5ad",
    "pre_res_HSCLSC": "adata_pre_res_HSCLSC.h5ad",
    "pre_nres_HSCLSClow": "adata_pre_nres_HSCLSClow.h5ad",
    "pre_nres_HSCLSChigh": "adata_pre_nres_HSCLSChigh.h5ad",
    "post_nres_HSCLSC": "adata_post_nres_HSCLSC.h5ad",
}
HSCs = {g: sc.read_h5ad(f) for g, f in HSC_files.items()}

MPP_files = {
    "pre_res_MPPCLP1": "adata_pre_res_MPPCLP1.h5ad",
    "pre_nres_MPPCLP1": "adata_pre_nres_MPPCLP1.h5ad",
    "post_nres_ASDC": "adata_post_nres_ASDC.h5ad",
    "post_nres_MPPpreB": "adata_post_nres_MPPpreB.h5ad",
    "post_nres_MPPCLP1": "adata_post_nres_MPPCLP1.h5ad",
}
MPPs = {g: sc.read_h5ad(f) for g, f in MPP_files.items()}

In [20]:
genes = [
    "ASS1",
    "ATF4",
    "ASNS",
    "ASL",
    "SLC7A1",
    "SLC7A2",
    "BCL2L1",
    "BCL11A",
    "NOTCH1",
    "NRIP1",
    "ATF1",
    "ATF3",
    "CCND3",
    "CDK6",
    "CREB1",
    "EGR1",
    "ERG",
    "FOS",
    "JUN",
    "JUNB",
    "JUND",
    "TPT1",
    "TPT1-AS1",
    "PRDX1",
    "AKT1",
    "AKT2",
    "AKT3",
    "ATG16L2",
    "DAPK1",
    "LRBA",
    "NCOA7",
    "NOL4L",
    "PRKACB",
    "CD74",
    "EEF1B2",
    "EIF1",
    "FAU",
    'DDIT3' #"CHOP",
]

In [21]:

import re
for gene in genes:
    if gene in adata.var_names:
        continue
    else:
        print(gene)
# print(genes in adata.var_names)

# pattern = r"^CD.$"

# akt_genes = adata.var_names[adata.var_names.str.match(pattern)]
# akt_genes.tolist()

In [22]:


def _get_expr_matrix(adata, use_layer=None, use_raw=False):
    """
    返回用于计算的表达矩阵 & 对应的 var_names
    - 默认用 adata.X
    - use_raw=True -> 用 adata.raw.X
    - use_layer="lognorm"/"counts" -> 用 adata.layers[use_layer]
    """
    if use_raw:
        if adata.raw is None:
            raise ValueError("use_raw=True but adata.raw is None")
        X = adata.raw.X
        var_names = adata.raw.var_names
    else:
        if use_layer is None:
            X = adata.X
        else:
            if use_layer not in adata.layers:
                raise ValueError(f"Layer '{use_layer}' not found in adata.layers: {list(adata.layers.keys())}")
            X = adata.layers[use_layer]
        var_names = adata.var_names
    return X, var_names


def mean_expression_for_genes(adata, genes, use_layer=None, use_raw=False):
    """
    返回：pd.Series(index=present_genes, value=mean_expr)
    """
    X, var_names = _get_expr_matrix(adata, use_layer=use_layer, use_raw=use_raw)

    present = [g for g in genes if g in var_names]
    missing = [g for g in genes if g not in var_names]

    if len(present) == 0:
        return pd.Series(dtype=float), missing

    # 取基因索引
    idx = [var_names.get_loc(g) for g in present]
    Xm = X[:, idx]

    # 稀疏/稠密都兼容
    if hasattr(Xm, "toarray"):
        Xm = Xm.toarray()

    mean_vals = Xm.mean(axis=0)
    return pd.Series(mean_vals, index=present, name="mean_expr"), missing


def compute_group_gene_means(adatas: dict, genes, use_layer=None, use_raw=False):
    """
    返回：
    - df_mean: rows=group, cols=genes_present_union
    - missing_map: {group: [missing_genes]}
    """
    rows = []
    missing_map = {}
    for group, ad in adatas.items():
        s, missing = mean_expression_for_genes(ad, genes, use_layer=use_layer, use_raw=use_raw)
        s.name = group
        rows.append(s)
        missing_map[group] = missing

    df_mean = pd.DataFrame(rows).sort_index()
    return df_mean, missing_map


# def log2fc_relative_to_baseline(df_mean, baseline_group, pseudocount=1e-6):
#     """
#     log2FC = log2( (mean + pc) / (baseline_mean + pc) )
#     """
#     if baseline_group not in df_mean.index:
#         raise ValueError(f"baseline_group '{baseline_group}' not in df_mean.index: {df_mean.index.tolist()}")

#     base = df_mean.loc[baseline_group]
#     df_log2fc = np.log2((df_mean + pseudocount) / (base + pseudocount))
#     df_log2fc.index.name = "group"
#     return df_log2fc
def ratio_relative_to_baseline(df_mean, baseline_group, pseudocount=1e-6):
    """
    ratio = (mean + pc) / (baseline_mean + pc)
    """
    if baseline_group not in df_mean.index:
        raise ValueError(f"baseline_group '{baseline_group}' not in df_mean.index")

    baseline = df_mean.loc[baseline_group]
    df_ratio = (df_mean + pseudocount) / (baseline + pseudocount)
    df_ratio.index.name = "group"
    return df_ratio
def combine_mean_and_ratio(df_mean, baseline_group, pseudocount=1e-6):
    """
    输入：
      df_mean: rows=group, cols=genes (mean expression)
    输出：
      df_combined: rows=group, cols=gene__mean, gene__ratio
    """
    if baseline_group not in df_mean.index:
        raise ValueError(f"baseline_group '{baseline_group}' not in df_mean.index")

    baseline = df_mean.loc[baseline_group]

    df_ratio = (df_mean + pseudocount) / (baseline + pseudocount)

    cols = []
    for g in df_mean.columns:
        cols.extend([
            (g, "mean"),
            (g, "ratio")
        ])

    df_combined = pd.DataFrame(
        index=df_mean.index,
        columns=pd.MultiIndex.from_tuples(cols)
    )

    for g in df_mean.columns:
        df_combined[(g, "mean")] = df_mean[g]
        df_combined[(g, "ratio")] = df_ratio[g]

    # 拉平成单层列名：GENE__mean / GENE__ratio
    df_combined.columns = [
        f"{gene}__{stat}" for gene, stat in df_combined.columns
    ]

    df_combined.index.name = "group"
    return df_combined



# ===== HSC: mean + relative to HC_HSC =====
# HSC_mean, HSC_missing = compute_group_gene_means(HSCs, genes, use_layer=None, use_raw=False)
# HSC_log2fc = log2fc_relative_to_baseline(HSC_mean, baseline_group="HC_HSC")

# # ===== MPP: mean + relative to pre_res_MPPCLP1 =====
# MPP_mean, MPP_missing = compute_group_gene_means(MPPs, genes, use_layer=None, use_raw=False)
# MPP_log2fc = log2fc_relative_to_baseline(MPP_mean, baseline_group="pre_res_MPPCLP1")
# ===== 输出 =====
# HSC_mean.to_csv("HSC_mean_expr_genes.csv")
# HSC_log2fc.to_csv("HSC_log2FC_vs_HC_HSC.csv")

# MPP_mean.to_csv("MPP_mean_expr_genes.csv")
# MPP_log2fc.to_csv("MPP_log2FC_vs_pre_res_MPPCLP1.csv")

# # ===== 可选：输出缺失基因报告 =====
# pd.DataFrame({"group": list(HSC_missing.keys()), "missing_genes": [",".join(v) for v in HSC_missing.values()]}).to_csv(
#     "HSC_missing_genes_report.csv", index=False
# )
# pd.DataFrame({"group": list(MPP_missing.keys()), "missing_genes": [",".join(v) for v in MPP_missing.values()]}).to_csv(
#     "MPP_missing_genes_report.csv", index=False
# )

# print("Done.")
# print("HSC mean:", HSC_mean.shape, "HSC log2FC:", HSC_log2fc.shape)
# print("MPP mean:", MPP_mean.shape, "MPP log2FC:", MPP_log2fc.shape)

HSC_mean, HSC_missing = compute_group_gene_means(
    HSCs, genes, use_layer=None, use_raw=False
)

HSC_mean_ratio = combine_mean_and_ratio(
    HSC_mean, baseline_group="HC_HSC"
)

HSC_mean_ratio.T.to_csv("HSC_mean_and_ratio_vs_HC_HSC.csv")
MPP_mean, MPP_missing = compute_group_gene_means(
    MPPs, genes, use_layer=None, use_raw=False
)

MPP_mean_ratio = combine_mean_and_ratio(
    MPP_mean, baseline_group="pre_res_MPPCLP1"
)

MPP_mean_ratio.T.to_csv("MPP_mean_and_ratio_vs_pre_res_MPPCLP1.csv")



In [18]:
0.462635851236537/0.935881990770345

0.4943313962647484